# Imports, API Data Request and PreProcessing

## Imports and Installations

In [3]:
!pip install requests

In [55]:
import requests
import json
import pandas as pd
import numpy as np
import concurrent.futures

## API Data Request

In [ ]:
""" potential data:
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")              
"""

    


In [17]:
import requests
import json
import pandas as pd
import concurrent.futures

# Ripple API endpoint
url = "https://testnet.xrpl-labs.com/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = response.json().get("result", {}).get("ledger_current_index")

# Define the number of ledgers to retrieve
num_ledgers = 1000

# Initialize list to store ledger data
ledger_data = []

# Function to retrieve ledger information
def get_ledger_data(ledger_index):
    print(f"Retrieving Ledger Index: {ledger_index}")

    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    response = requests.post(url, json=payload)

    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})
        data = []

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    # General transaction fields
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "Sequence": tx.get("Sequence"),
                    "AccountTxnID": tx.get("AccountTxnID"),
                    "Flags": tx.get("Flags"),
                    "LastLedgerSequence": tx.get("LastLedgerSequence"),
                    "Memos": tx.get("Memos"),
                    "NetworkID": tx.get("NetworkID"),
                    "Signers": tx.get("Signers"),
                    "SourceTag": tx.get("SourceTag"),
                    "SigningPubKey": tx.get("SigningPubKey"),
                    "TicketSequence": tx.get("TicketSequence"),
                    "TxnSignature": tx.get("TxnSignature"),
                    # Payment-specific fields
                    "Amount": tx.get("Amount"),
                    "Destination": tx.get("Destination"),
                    "DestinationTag": tx.get("DestinationTag"),
                    "InvoiceID": tx.get("InvoiceID"),
                    "Paths": tx.get("Paths"),
                    "SendMax": tx.get("SendMax"),
                    "DeliverMin": tx.get("DeliverMin"),
                    # OfferCreate-specific fields
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Expiration": tx.get("Expiration"),
                    "OfferSequence": tx.get("OfferSequence"),
                    # TrustSet-specific fields
                    "LimitAmount": tx.get("LimitAmount"),
                    "QualityIn": tx.get("QualityIn"),
                    "QualityOut": tx.get("QualityOut"),
                    # Escrow-specific fields
                    "FinishAfter": tx.get("FinishAfter"),
                    "CancelAfter": tx.get("CancelAfter"),
                    "Condition": tx.get("Condition"),
                    "Fulfillment": tx.get("Fulfillment"),
                    # Meta-information
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "AffectedNodes": tx.get("meta", {}).get("AffectedNodes"),
                    "DeliveredAmount": tx.get("meta", {}).get("delivered_amount"),
                    # Additional fields for other transaction types
                    "OwnerCount": tx.get("OwnerCount"),
                    "SetFlag": tx.get("SetFlag"),
                    "ClearFlag": tx.get("ClearFlag"),
                    "Domain": tx.get("Domain"),
                    "EmailHash": tx.get("EmailHash"),
                    "MessageKey": tx.get("MessageKey"),
                    "TransferRate": tx.get("TransferRate"),
                    "RegularKey": tx.get("RegularKey")
                })
        return data

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)
        return []

# Use ThreadPoolExecutor to parallelize ledger retrieval
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_ledger_data, str(int(latest_ledger_index) - i)) for i in range(num_ledgers)]

    # Collect results as they complete
    for future in concurrent.futures.as_completed(futures):
        ledger_data.extend(future.result())  # Add the results to the ledger_data list

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)
print(df.head())


Retrieving Ledger Index: 764316Retrieving Ledger Index: 764315

Retrieving Ledger Index: 764314
Retrieving Ledger Index: 764313
Retrieving Ledger Index: 764312
Retrieving Ledger Index: 764311
Retrieving Ledger Index: 764310
Retrieving Ledger Index: 764309
Retrieving Ledger Index: 764308
Retrieving Ledger Index: 764307
Retrieving Ledger Index: 764306
Retrieving Ledger Index: 764305
Retrieving Ledger Index: 764304
Retrieving Ledger Index: 764303
Retrieving Ledger Index: 764302
Retrieving Ledger Index: 764301
Retrieving Ledger Index: 764300
Retrieving Ledger Index: 764299
Retrieving Ledger Index: 764298
Retrieving Ledger Index: 764297
Retrieving Ledger Index: 764296
Retrieving Ledger Index: 764295
Retrieving Ledger Index: 764294
Retrieving Ledger Index: 764293
Retrieving Ledger Index: 764292
Retrieving Ledger Index: 764291
Retrieving Ledger Index: 764290
Retrieving Ledger Index: 764289
Retrieving Ledger Index: 764288
Retrieving Ledger Index: 764287
Retrieving Ledger Index: 764286
Retrievi

## Data Preprocessing

In [18]:
df

,LedgerSequence,CloseTime,TransactionType,Account,Fee,Sequence,AccountTxnID,Flags,LastLedgerSequence,Memos,...,AffectedNodes,DeliveredAmount,OwnerCount,SetFlag,ClearFlag,Domain,EmailHash,MessageKey,TransferRate,RegularKey
0,764316,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253555,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,None,NaN,None,None,None,None,None,None
1,764316,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253557,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,None,NaN,None,None,None,None,None,None
2,764316,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253558,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,None,NaN,None,None,None,None,None,None
3,764316,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253559,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,None,NaN,None,None,None,None,None,None
4,764316,None,Payment,rfydDYbhxkrwoffCuYzAatVJ1NGuMyRKfF,10,413053,None,NaN,765100.0,[{'Memo': {'MemoData': '4642505266410002000000...,...,None,None,None,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,763318,2024-Sep-16 13:59:30.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250512,None,8.000000e+00,763336.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,None,NaN,None,None,None,None,None,None
4018,763317,2024-Sep-16 13:59:23.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250508,None,8.000000e+00,763335.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,None,NaN,None,None,None,None,None,None
4019,763317,2024-Sep-16 13:59:23.000000000 UTC,Payment,rGGfbHgqVAUXa4XAEUSYgondevrR69xSUP,15,763311,None,2.147484e+09,766399.0,None,...,None,None,None,NaN,None,None,None,None,None,None
4020,763317,2024-Sep-16 13:59:23.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250510,None,8.000000e+00,763335.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,None,NaN,None,None,None,None,None,None


In [19]:
df.dtypes

LedgerSequence         object
CloseTime              object
TransactionType        object
Account                object
Fee                    object
Sequence                int64
AccountTxnID           object
Flags                 float64
LastLedgerSequence    float64
Memos                  object
NetworkID              object
Signers                object
SourceTag             float64
SigningPubKey          object
TicketSequence        float64
TxnSignature           object
Amount                 object
Destination            object
DestinationTag        float64
InvoiceID              object
Paths                  object
SendMax                object
DeliverMin             object
TakerGets              object
TakerPays              object
Expiration             object
OfferSequence         float64
LimitAmount            object
QualityIn              object
QualityOut             object
FinishAfter            object
CancelAfter            object
Condition              object
Fulfillmen

In [20]:
# Convert general fields
df['LedgerSequence'] = pd.to_numeric(df['LedgerSequence'])
df['CloseTime'] = pd.to_datetime(df['CloseTime'])
df['TransactionType'] = df['TransactionType'].astype(str)
df['Account'] = df['Account'].astype(str)
df['Fee'] = pd.to_numeric(df['Fee'])
df['Sequence'] = pd.to_numeric(df['Sequence'])
df['AccountTxnID'] = df['AccountTxnID'].astype(str)

# Convert numeric fields
df['Flags'] = pd.to_numeric(df['Flags'], errors='coerce')
df['LastLedgerSequence'] = pd.to_numeric(df['LastLedgerSequence'], errors='coerce')
df['NetworkID'] = pd.to_numeric(df['NetworkID'], errors='coerce')
df['SourceTag'] = pd.to_numeric(df['SourceTag'], errors='coerce')
df['TicketSequence'] = pd.to_numeric(df['TicketSequence'], errors='coerce')
df['OfferSequence'] = pd.to_numeric(df['OfferSequence'], errors='coerce')
df['OwnerCount'] = pd.to_numeric(df['OwnerCount'], errors='coerce')
df['DestinationTag'] = pd.to_numeric(df['DestinationTag'], errors='coerce')
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')  # If 'Amount' is always numeric
df['SendMax'] = pd.to_numeric(df['SendMax'], errors='coerce')  # If 'SendMax' is numeric
df['DeliverMin'] = pd.to_numeric(df['DeliverMin'], errors='coerce')
df['Expiration'] = pd.to_numeric(df['Expiration'], errors='coerce')
df['LimitAmount'] = pd.to_numeric(df['LimitAmount'], errors='coerce')
df['QualityIn'] = pd.to_numeric(df['QualityIn'], errors='coerce')
df['QualityOut'] = pd.to_numeric(df['QualityOut'], errors='coerce')
df['TransferRate'] = pd.to_numeric(df['TransferRate'], errors='coerce')
df['RegularKey'] = pd.to_numeric(df['RegularKey'], errors='coerce')  # If applicable

# Convert datetime fields
df['FinishAfter'] = pd.to_datetime(df['FinishAfter'], errors='coerce')
df['CancelAfter'] = pd.to_datetime(df['CancelAfter'], errors='coerce')

# Convert string fields
df['SigningPubKey'] = df['SigningPubKey'].astype(str)
df['TxnSignature'] = df['TxnSignature'].astype(str)
df['InvoiceID'] = df['InvoiceID'].astype(str)
df['Destination'] = df['Destination'].astype(str)  # Assuming Destination is always an address format
df['Domain'] = df['Domain'].astype(str)  # Domain is a string field
df['EmailHash'] = df['EmailHash'].astype(str)  # EmailHash is a string field
df['MessageKey'] = df['MessageKey'].astype(str)  # MessageKey is a string field
df['Condition'] = df['Condition'].astype(str)  # Assuming Condition is a string field
df['Fulfillment'] = df['Fulfillment'].astype(str)  # Assuming Fulfillment is a string field
df['TransactionResult'] = df['TransactionResult'].astype(str)

# Convert object fields (list of dictionaries or complex structures)
df['Memos'] = df['Memos'].astype(object)  # Treat 'Memos' as an object (e.g., list of dicts)
df['Signers'] = df['Signers'].astype(object)
df['Paths'] = df['Paths'].astype(object)
df['TakerGets'] = df['TakerGets'].astype(object)  # 'TakerGets' and 'TakerPays' can be either dicts or strings
df['TakerPays'] = df['TakerPays'].astype(object)
df['AffectedNodes'] = df['AffectedNodes'].astype(object)  # AffectedNodes is a complex structure

# Special cases
# Handle 'TakerGets' and 'TakerPays' since they can be either numeric or objects (e.g., dicts)
df['TakerGets'] = df['TakerGets'].apply(lambda x: str(x) if isinstance(x, dict) else pd.to_numeric(x, errors='coerce'))
df['TakerPays'] = df['TakerPays'].apply(lambda x: str(x) if isinstance(x, dict) else pd.to_numeric(x, errors='coerce'))

# Convert boolean or flag fields
df['SetFlag'] = pd.to_numeric(df['SetFlag'], errors='coerce')
df['ClearFlag'] = pd.to_numeric(df['ClearFlag'], errors='coerce')

# Replace the string "None" with real None (NaN in pandas)
df = df.replace("None", None)


In [21]:
df.dtypes

LedgerSequence                      int64
CloseTime             datetime64[ns, UTC]
TransactionType                    object
Account                            object
Fee                                 int64
Sequence                            int64
AccountTxnID                       object
Flags                             float64
LastLedgerSequence                float64
Memos                              object
NetworkID                         float64
Signers                            object
SourceTag                         float64
SigningPubKey                      object
TicketSequence                    float64
TxnSignature                       object
Amount                            float64
Destination                        object
DestinationTag                    float64
InvoiceID                          object
Paths                              object
SendMax                           float64
DeliverMin                        float64
TakerGets                         

In [60]:
df

,LedgerSequence,CloseTime,TransactionType,Account,Fee,Sequence,AccountTxnID,Flags,LastLedgerSequence,Memos,...,AffectedNodes,DeliveredAmount,OwnerCount,SetFlag,ClearFlag,Domain,EmailHash,MessageKey,TransferRate,RegularKey
0,764316,NaT,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253555,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
1,764316,NaT,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253557,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
2,764316,NaT,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253558,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
3,764316,NaT,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2253559,None,8.000000e+00,764334.0,[{'Memo': {'MemoData': '3638303033365F34333339...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
4,764316,NaT,Payment,rfydDYbhxkrwoffCuYzAatVJ1NGuMyRKfF,10,413053,None,NaN,765100.0,[{'Memo': {'MemoData': '4642505266410002000000...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,763318,2024-09-16 13:59:30+00:00,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250512,None,8.000000e+00,763336.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
4018,763317,2024-09-16 13:59:23+00:00,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250508,None,8.000000e+00,763335.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
4019,763317,2024-09-16 13:59:23+00:00,Payment,rGGfbHgqVAUXa4XAEUSYgondevrR69xSUP,15,763311,None,2.147484e+09,766399.0,None,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN
4020,763317,2024-09-16 13:59:23+00:00,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,2250510,None,8.000000e+00,763335.0,[{'Memo': {'MemoData': '3638303033365F34333631...,...,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN


In [56]:
# check if API request worked for every ledger
np.sort(df['LedgerSequence'].unique())

array([763317, 763318, 763319, 763320, 763321, 763322, 763323, 763324,
       763325, 763326, 763327, 763328, 763329, 763330, 763331, 763332,
       763333, 763334, 763335, 763336, 763337, 763338, 763339, 763340,
       763341, 763342, 763343, 763344, 763345, 763346, 763347, 763348,
       763349, 763350, 763351, 763352, 763353, 763354, 763355, 763356,
       763357, 763358, 763359, 763360, 763361, 763362, 763363, 763364,
       763365, 763366, 763367, 763368, 763369, 763370, 763371, 763372,
       763373, 763374, 763379, 763381, 763382, 763383, 763384, 763385,
       763386, 763387, 763388, 763389, 763390, 763391, 763392, 763393,
       763394, 763395, 763396, 763397, 763398, 763399, 763400, 763401,
       763402, 763403, 763404, 763405, 763406, 763407, 763408, 763409,
       763410, 763411, 763412, 763413, 763414, 763415, 763416, 763417,
       763418, 763419, 763420, 763421, 763422, 763423, 763424, 763425,
       763426, 763427, 763428, 763429, 763430, 763431, 763432, 763433,
      

In [23]:
# get overview
df.describe()

,LedgerSequence,Fee,Sequence,Flags,LastLedgerSequence,NetworkID,SourceTag,TicketSequence,Amount,DestinationTag,...,LimitAmount,QualityIn,QualityOut,FinishAfter,CancelAfter,OwnerCount,SetFlag,ClearFlag,TransferRate,RegularKey
count,4022.000000,4022.000000,4.022000e+03,3.743000e+03,4020.000000,0.0,3.057000e+03,169.000000,9.100000e+02,3.950000e+02,...,0.0,0.0,0.0,0,0,0.0,2.0,0.0,0.0,0.0
mean,763785.153158,25.548235,1.797302e+06,1.772857e+08,764396.040299,NaN,3.961179e+07,220417.461538,6.663679e+07,1.381826e+09,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
min,763317.000000,10.000000,0.000000e+00,0.000000e+00,763326.000000,NaN,3.888739e+07,16853.000000,1.000000e+02,1.000000e+00,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
25%,763526.000000,10.000000,2.250543e+06,8.000000e+00,763569.000000,NaN,3.888739e+07,230445.000000,4.000000e+06,6.000877e+07,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
50%,763756.000000,10.000000,2.251548e+06,8.000000e+00,763821.500000,NaN,3.888739e+07,230515.000000,3.996000e+07,6.837346e+08,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
75%,764072.000000,10.000000,2.252554e+06,8.000000e+00,764159.000000,NaN,3.888739e+07,230598.000000,9.000000e+07,2.897967e+09,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
max,764316.000000,30000.000000,2.253559e+06,2.148008e+09,789929.000000,NaN,6.588793e+08,230681.000000,1.363400e+09,4.258789e+09,...,NaN,NaN,NaN,NaT,NaT,NaN,4.0,NaN,NaN,NaN
std,292.028874,504.572858,8.202721e+05,5.910886e+08,3350.488310,NaN,1.869805e+07,45497.849470,1.425487e+08,1.483871e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [37]:
# check for dublicates
df[(df['Sequence'] > 0) & (df[['Account', 'Sequence']].duplicated(keep=False))]

,LedgerSequence,CloseTime,TransactionType,Account,Fee,Sequence,AccountTxnID,Flags,LastLedgerSequence,Memos,...,AffectedNodes,DeliveredAmount,OwnerCount,SetFlag,ClearFlag,Domain,EmailHash,MessageKey,TransferRate,RegularKey


In [59]:
# check percentage of missing values for each column
df.isnull().mean()

LedgerSequence        0.000000
CloseTime             0.001492
TransactionType       0.000000
Account               0.000000
Fee                   0.000000
Sequence              0.000000
AccountTxnID          1.000000
Flags                 0.069368
LastLedgerSequence    0.000497
Memos                 0.136499
NetworkID             1.000000
Signers               0.999005
SourceTag             0.239930
SigningPubKey         0.000000
TicketSequence        0.957981
TxnSignature          0.000995
Amount                0.773744
Destination           0.771258
DestinationTag        0.901790
InvoiceID             1.000000
Paths                 1.000000
SendMax               1.000000
DeliverMin            1.000000
TakerGets             0.991298
TakerPays             0.991298
Expiration            1.000000
OfferSequence         0.991795
LimitAmount           1.000000
QualityIn             1.000000
QualityOut            1.000000
FinishAfter           1.000000
CancelAfter           1.000000
Conditio